# Pull Video Duration using Youtube API

In [42]:
import pandas as pd
from tqdm import tqdm
from googleapiclient.discovery import build

with open("/home/robmulla/Repos/data-science-youtube-stats/apikey/youtube.key") as f:
    youTubeApiKey = f.read()
youtube = build("youtube", "v3", developerKey=youTubeApiKey)

In [2]:
df = pd.read_csv('youtube-trending-video-dataset/US_youtube_trending_data.csv')

In [43]:
video_ids = df['video_id'].unique()

In [70]:
stats = []
for i in tqdm(range(0, len(video_ids), 40), total=455):
    res = (
        (youtube)
        .videos()
        .list(id=",".join(video_ids[i : i + 40]),
              part="snippet,statistics,contentDetails,status,topicDetails,recordingDetails,localizations",
             )
        .execute()
    )
    stats += res["items"]
    # break
stats_df = pd.json_normalize(stats)
df_stats = df_column_uniquify(df)

100%|██████████| 455/455 [02:06<00:00,  3.60it/s]


In [73]:
durations_df = pd.json_normalize(stats)[['id','contentDetails.duration']]

In [74]:
durations_df.head()

,id,contentDetails.duration
0,z_LdKL0drkA,PT13M30S
1,1niAIYz6JZg,PT12M29S
2,jTMJK7wb0rM,PT1H3M18S
3,gHPubcpGkrk,PT9M40S
4,UUOGVgComrU,PT4M8S


In [75]:
durations_df["duration"] = durations_df["contentDetails.duration"].apply(pd.Timedelta)
durations_df["duration_seconds"] = (
    durations_df["duration"].astype("timedelta64[s]").fillna(0).astype("int")
)


In [76]:
durations_df.head()

,id,contentDetails.duration,duration,duration_seconds
0,z_LdKL0drkA,PT13M30S,0 days 00:13:30,810
1,1niAIYz6JZg,PT12M29S,0 days 00:12:29,749
2,jTMJK7wb0rM,PT1H3M18S,0 days 01:03:18,3798
3,gHPubcpGkrk,PT9M40S,0 days 00:09:40,580
4,UUOGVgComrU,PT4M8S,0 days 00:04:08,248


In [77]:
durations_df.head()

,id,contentDetails.duration,duration,duration_seconds
0,z_LdKL0drkA,PT13M30S,0 days 00:13:30,810
1,1niAIYz6JZg,PT12M29S,0 days 00:12:29,749
2,jTMJK7wb0rM,PT1H3M18S,0 days 01:03:18,3798
3,gHPubcpGkrk,PT9M40S,0 days 00:09:40,580
4,UUOGVgComrU,PT4M8S,0 days 00:04:08,248


In [82]:
durations_df.to_csv('durations.csv', index=False)